# TAIGA IACT reconstruction analysis

This notebook evaluates the likelihood-based reconstruction by comparing
the recovered shower parameters with the ground truth values provided in
the TAIGA test sample.  Before executing the cells ensure that
`run_reconstruction_from_config` has been called once so that the
likelihood model cached at `model_path` exists.  The configuration file
controls which template and test events are processed, making it easy to
trade accuracy for runtime when exploring larger datasets.


In [ ]:
from pathlib import Path
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from TAIGA_IACT_likelihood import (
    TAIGADataset,
    load_configuration,
    load_model_package,
    reconstruct_dataset,
)


In [ ]:
CONFIG_PATH = Path('config/reconstruction_config.json')
config = load_configuration(CONFIG_PATH)
model_path = Path(config['model_path']).expanduser().resolve()
model, bounds = load_model_package(model_path)

def matching_csv_path(txt_path: Path) -> Path:
    name = txt_path.name.replace('_clean_', '_hillas_')
    stem, _ = name.rsplit('.', 1)
    return txt_path.with_name(f"{stem}.csv")

test_txt = Path(config['test_txt_file']).expanduser().resolve()
test_csv = matching_csv_path(test_txt)
test_dataset = TAIGADataset(test_txt, test_csv)

seed = config.get('random_seed')
rng = random.Random(seed) if seed is not None else random.Random()

results = reconstruct_dataset(
    model,
    bounds,
    test_dataset,
    rng=rng,
    n_samples=int(config.get('n_samples', 4096)),
    refine_iterations=int(config.get('refine_iterations', 4)),
    refine_radius=float(config.get('refine_radius', 0.1)),
    limit=config.get('test_event_limit'),
)
results_df = pd.DataFrame(results)
results_df.head()


## Energy reconstruction accuracy

We bin the true shower energy into ten logarithmic intervals between
20 and 200 TeV and report the average fractional error (in percent)
within each bin.

In [ ]:
energy_true = results_df['energy_true'].to_numpy()
energy_est = results_df['energy_est'].to_numpy()
energy_error_pct = 100.0 * np.abs(energy_est - energy_true) / energy_true

bins = np.logspace(np.log10(20.0), np.log10(200.0), num=11)
bin_indices = np.digitize(energy_true, bins) - 1
bin_centers = np.sqrt(bins[:-1] * bins[1:])
avg_error = []
for idx in range(10):
    mask = bin_indices == idx
    if not np.any(mask):
        avg_error.append(np.nan)
    else:
        avg_error.append(energy_error_pct[mask].mean())

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(bin_centers, avg_error, marker='o')
ax.set_xscale('log')
ax.set_xlabel('True energy [TeV]')
ax.set_ylabel('Average |ΔE| / E [%]')
ax.set_title('Energy reconstruction accuracy by energy bin')
ax.grid(True, which='both', ls=':')
plt.show()

## Axis and direction accuracy

We inspect the ground impact position residual and the angular
separation between reconstructed and true arrival directions.

In [ ]:
dx = results_df['x_ground_est'] - results_df['x_ground_true']
dy = results_df['y_ground_est'] - results_df['y_ground_true']
axis_error = np.sqrt(dx**2 + dy**2)

def to_unit_vector(theta, phi):
    sin_t = np.sin(theta)
    return np.stack((sin_t * np.cos(phi), sin_t * np.sin(phi), np.cos(theta)), axis=-1)

vec_true = to_unit_vector(results_df['source_tet_true'].to_numpy(), results_df['source_fi_true'].to_numpy())
vec_est = to_unit_vector(results_df['source_tet_est'].to_numpy(), results_df['source_fi_est'].to_numpy())
dot = np.einsum('ij,ij->i', vec_true, vec_est)
dot = np.clip(dot, -1.0, 1.0)
angular_error = np.degrees(np.arccos(dot))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharex=True)
ax1.scatter(results_df['energy_true'], axis_error, alpha=0.6)
ax1.set_xscale('log')
ax1.set_xlabel('True energy [TeV]')
ax1.set_ylabel('Ground impact error [m]')
ax1.set_title('Axis reconstruction error')
ax1.grid(True, which='both', ls=':')

ax2.scatter(results_df['energy_true'], angular_error, alpha=0.6, color='C1')
ax2.set_xscale('log')
ax2.set_xlabel('True energy [TeV]')
ax2.set_ylabel('Angular separation [deg]')
ax2.set_title('Arrival direction error')
ax2.grid(True, which='both', ls=':')
plt.tight_layout()
plt.show()